In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from Utils.DataLoader import DataLoader
from Utils.DataTransformer import DataTransformer
from Utils.DuckDb.DuckDb import DuckDb
from Utils.DatasetProcessor.DatasetProcessorUtils import DatasetProcessorUtils
from Utils.DataAnalysis import DataAnalysis
from datetime import datetime
import pandas as pd
from typing import List
from Utils import PandasNotebookConfigs
import project_config
import numpy as np
import matplotlib.pyplot as plt
import joblib
from datetime import datetime
from Utils.Datetime import DatetimeUtils
from Utils.Dict import DictUtils
from Utils.Number import NumberUtils

In [11]:
PandasNotebookConfigs.config()

Ajustando display.float.format para %.4f
Ajustando prints de linhas e colunas


In [ ]:
# 50% of the users will renew the plan
SUCESS_USER_PROACTIVE_ACTION = 0.50

In [12]:
DUCK_DB = DuckDb()
DATA_TRANSFORMER = DataTransformer()
DATA_PROCESSOR_UTILS = DatasetProcessorUtils(DUCK_DB)
DATETIME_UTILS = DatetimeUtils()
DICT_UTILS = DictUtils()
DATA_ANALYSIS = DataAnalysis()
NUMBER_UTILS = NumberUtils()

In [13]:
df = DUCK_DB.load_table(
    project_config.PREDICTED_DATA_TABLE
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [14]:
len(df)

1251070

In [27]:
# Get only data to predict
df = df[df['no_churn_information'] == True]
len(df)

590951

In [30]:
mean_price = df['price_per_month'].mean()
print(f'Preço médio pago por mês: ${mean_price:.2f}')

Preço médio pago por mês: $133.63


In [31]:
predicted_is_churn_df = df[df['predicted_is_churn'] == True]
predicted_not_is_churn_df = df[df['predicted_is_churn'] == False]

len(predicted_is_churn_df), len(predicted_not_is_churn_df)

(306184, 284767)

In [ ]:
proactive_action_earn_value = (len(predicted_is_churn_df) / 2) * mean_price
print(f'Valor total arrecadado pela ação proativa: ${NUMBER_UTILS.format_float_number(proactive_action_earn_value)}')

Valor total arrecadado pela ação proativa: $204.572.27..159.667.496


In [38]:
str(100.02)

'100.02'